<a href="https://colab.research.google.com/github/aggelospsiris/NB-KNN-SMV/blob/main/NBvsSMVvsKNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from math import sqrt
import warnings
warnings.filterwarnings('ignore')
from google.colab import drive 
drive.mount('/drive')
df=pd.read_csv('/drive/My Drive/Colab Notebooks/decision_theory/processed_data.csv')

Drive already mounted at /drive; to attempt to forcibly remount, call drive.mount("/drive", force_remount=True).


In [ ]:
df = df.drop(columns=['Unnamed: 0'])

In [ ]:
# Split into input and output
X = df.values[:, :-1]
Y = df.values[:, -1]

In [ ]:
#normalization between [-1 1]
#a = np.array(X)
#X = 2.*(a - np.min(a))/np.ptp(a)-1

In [ ]:
#better results normalizing the data by removing the mean and scaling to unit variance
from sklearn import preprocessing 
scaler = preprocessing.StandardScaler().fit(X)
X = scaler.transform(X)

In [ ]:
from sklearn.model_selection import KFold
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn import metrics

# Split the data to training and testing data 5-Fold
kfold = KFold(n_splits=5, shuffle=True)
#The dataframe that my metric will be saved
df_metrics_NB = pd.DataFrame(columns=['Fold','Specificity','Sensitivity'])
df_metrics_SVM= pd.DataFrame(columns=['Fold','Specificity','Sensitivity'])
df_metrics_KNN= pd.DataFrame(columns=['Fold','Specificity','Sensitivity'])

#models
gnb = GaussianNB()
clf_svm = SVC(C=21,gamma=0.5,kernel='rbf')
knn = KNeighborsClassifier(n_neighbors=9,metric='l1')

for i, (train, test) in enumerate(kfold.split(X)):
  # fit the model
  gnb.fit(X[train], Y[train])
  clf_svm.fit(X[train], Y[train])
  knn.fit(X[train], Y[train])
  #predict
  y_pred_NB = gnb.predict(X[test])
  y_pred_SVC = clf_svm.predict(X[test])
  y_pred_KNN = knn.predict(X[test])
  #colnfusion matrix
  cm_nb = metrics.confusion_matrix(Y[test],y_pred_NB)
  cm_svm = metrics.confusion_matrix(Y[test],y_pred_SVC)
  cm_knn = metrics.confusion_matrix(Y[test],y_pred_KNN)
  #calculate Accuracy,Specificity,Sensitivity
  #nb
  total_nb=sum(sum(cm_nb))
  Specificity_nb = cm_nb[0,0]/(cm_nb[0,0]+cm_nb[0,1])
  Sensitivity_nb = cm_nb[1,1]/(cm_nb[1,0]+cm_nb[1,1])
  #svm
  total_svm=sum(sum(cm_svm))
  Specificity_svm = cm_svm[0,0]/(cm_svm[0,0]+cm_svm[0,1])
  Sensitivity_svm = cm_svm[1,1]/(cm_svm[1,0]+cm_svm[1,1])
  #knn
  total_knn=sum(sum(cm_knn))
  Specificity_knn = cm_knn[0,0]/(cm_knn[0,0]+cm_knn[0,1])
  Sensitivity_knn = cm_knn[1,1]/(cm_knn[1,0]+cm_knn[1,1])
  #apend values to the dataframes
  df_metrics_NB.loc[len(df_metrics_NB.index)] = [i,Specificity_nb,Sensitivity_nb] 
  df_metrics_SVM.loc[len(df_metrics_SVM.index)] = [i,Specificity_svm,Sensitivity_svm] 
  df_metrics_KNN.loc[len(df_metrics_KNN.index)] = [i, Specificity_knn,Sensitivity_knn] 

#Calculate the mean of the metrics
GeometricMean_NB = sqrt (df_metrics_NB['Specificity'].mean()*df_metrics_NB['Sensitivity'].mean())
GeometricMean_SVM = sqrt (df_metrics_SVM['Specificity'].mean()*df_metrics_SVM['Sensitivity'].mean())
GeometricMean_KNN = sqrt (df_metrics_KNN['Specificity'].mean()*df_metrics_KNN['Sensitivity'].mean())
print("GeometricMean(SVM):",GeometricMean_SVM)
print("GeometricMean(NB):",GeometricMean_NB)
print("GeometricMean(KNN):",GeometricMean_KNN)



GeometricMean(SVM): 0.5837094854745561
GeometricMean(NB): 0.6212663848947874
GeometricMean(KNN): 0.4730653907674738


In [ ]:
df_metrics_NB

,Fold,Specificity,Sensitivity
0,0.0,0.352273,1.000000
1,1.0,0.430380,0.947368
2,2.0,0.409639,0.970588
3,3.0,0.380952,0.937500
4,4.0,0.463415,0.882353


In [ ]:
df_metrics_SVM

,Fold,Specificity,Sensitivity
0,0.0,0.750000,0.379310
1,1.0,0.784810,0.500000
2,2.0,0.795181,0.382353
3,3.0,0.726190,0.437500
4,4.0,0.817073,0.500000


In [ ]:
df_metrics_KNN

,Fold,Specificity,Sensitivity
0,0.0,0.750000,0.241379
1,1.0,0.873418,0.394737
2,2.0,0.831325,0.205882
3,3.0,0.797619,0.281250
4,4.0,0.865854,0.235294
